In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

In [32]:
import torch.nn as nn
from metal.mmtl.task import Task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo = Task("foo_task", [make_dataloader(5000), None, None], foo_input, foo_head)
bar = Task("bar_task", [make_dataloader(2000), None, None], bar_input, bar_head)
# baz = Task("baz_task", "baz_head", [make_dataloader(100), None, None])
tasks = [foo]

In [40]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(model, tasks, n_epochs=10, lr=0.01, progress_bar=True)

[1 epo]:
[2 epo]:
[3 epo]:
[4 epo]:
[5 epo]:
[6 epo]:
[7 epo]:
[8 epo]:
[9 epo]:
[10 epo]:


In [28]:
for batch in foo.data_loaders[0]:
    X, Y = batch
    print(model(X, ['foo_task']))
    print(model.calculate_loss(X, Y, ['foo_task']))    
    print(model.calculate_output(X, ['foo_task']))    
    break

{'foo_task': tensor([[-0.2317, -0.1599],
        [-0.2323,  0.0496],
        [-0.1185, -0.1865],
        [-0.2607, -0.0905]], grad_fn=<AddmmBackward>)}
{'foo_task': tensor(0.6408, grad_fn=<NllLossBackward>)}
{'foo_task': tensor([[0.4821, 0.5179],
        [0.4300, 0.5700],
        [0.5170, 0.4830],
        [0.4575, 0.5425]])}
